# Usuarios y títulos

In [1]:
import ast
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Préstamos

In [2]:
%%time

merge_df = pd.read_csv('merge.csv', header=0)
merge_df = merge_df.fillna('')

CPU times: user 1.41 s, sys: 67.4 ms, total: 1.47 s
Wall time: 1.47 s


In [3]:
merge_df.shape

(516206, 12)

In [4]:
merge_df.head()

,TITULO,AUTOR,DEWEY,FECHA_PUB,PAIS_EJEMPLAR,COD_BARRAS_LECTOR,CODBAR,Sucursal,Tratamiento,Edad,Ciudad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",925,2010,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",843,2009,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",986,2009,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49
3,Maravillas del mundo,"Steele, Philip.",910,2007,Mexico,88007424,88007424,ARM,sr.,43,Armenia,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",972,2008,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49


## Thresholds

In [5]:
THRESHOLD_GROUPS = 100

In [6]:
THRESHOLD_TITLES = 5

In [35]:
TOP = 10

## Groups

In [8]:
group_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad']).CODBAR.count()

In [9]:
len(group_count_sr)

333

In [10]:
filtered_group_count_sr = group_count_sr.sort_values(ascending=False)[lambda x: x >= THRESHOLD_GROUPS]

In [11]:
len(filtered_group_count_sr)

245

In [12]:
filtered_group_count_sr.head()

Ciudad  Tratamiento  Rango_edad
Bogotá  sr.          18-29         46986
        sra.         18-29         42716
        sr.          30-39         31544
Tunja   sra.         -17           26232
Bogotá  sra.         30-39         22022
Name: CODBAR, dtype: int64

## Titles

In [13]:
title_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad', 'TITULO']).CODBAR.count()

In [14]:
len(title_count_sr)

321402

In [15]:
filtered_title_count_sr = title_count_sr.sort_values(ascending=False)[lambda x: x >= THRESHOLD_TITLES]

In [16]:
len(filtered_title_count_sr)

12853

In [17]:
filtered_title_count_sr.head()

Ciudad  Tratamiento  Rango_edad  TITULO                             
Bogotá  sr.          18-29       Obras completas                        118
        sra.         18-29       Obras completas                         72
        sr.          30-39       Obras completas                         67
Neiva   sr.          18-29       Principios de anatomía y fisiología     60
Tunja   sra.         -17         Orejitas                                54
Name: CODBAR, dtype: int64

## Count

In [18]:
ciudades = merge_df.Ciudad.unique()
tratamientos = merge_df.Tratamiento.unique()
rangos_edades = merge_df.Rango_edad.unique()

In [36]:
%%time

indexes = []
for ciudad in ciudades:
    for tratamiento in tratamientos:
        for rango_edad in rangos_edades:
            try:
                index = (ciudad, tratamiento, rango_edad)
                if index in group_count_sr:
                    sr = title_count_sr[index]
                    sorted_sr = sr.sort_values(ascending=False).head(TOP)
                    for title in sorted_sr.index:
                        indexes.append(index + (title,))
            except:
                pass

CPU times: user 562 ms, sys: 0 ns, total: 562 ms
Wall time: 562 ms


In [37]:
selected_title_count_sr = title_count_sr[indexes]

In [39]:
selected_title_count_df = pd.DataFrame(selected_title_count_sr)

In [40]:
%%time

def process_group_count(x):
    index = x.name[:3]
    count = group_count_sr[index]
    return count

selected_title_count_df['Cuenta_grupo'] = selected_title_count_df.apply(process_group_count, axis=1)
selected_title_count_df = selected_title_count_df.rename(columns={'CODBAR': 'Cuenta_titulo'})

CPU times: user 304 ms, sys: 31 µs, total: 304 ms
Wall time: 302 ms


In [42]:
selected_title_count_df

Cuenta_titulo  \
Ciudad  Tratamiento Rango_edad TITULO                                                              
Armenia sr.         40-49      Que nos tengan en cuenta : colonos, empresarios...              4   
                               El hombre duplicado                                             4   
                               Los secretos de las obras de arte                               3   
                               Colonización y protesta campesina en Colombia (...              3   
                               El Conde de Montecristo                                         3   
                               Desvíos de lo real : el cine de no ficción                      3   
                               Luis Vidales y la crítica de arte en Colombia                   3   
                               El principito                                                   3   
                               Distribución en planta                                          2   
                               El blog de Luis Sepúlveda                                       2   
                    -17        Calvin y hobbes                                                15   
                               Vuelo 714 para Sidney                                          13   
                               Tiranosaurio : lagarto tirano                                  11   
                               Los cigarros del faraón                                        11   
                               ¿Dónde está Wally?                                             11   
                               El templo del sol                                              10   
                               Encuentra los animales.                                        10   
                               Gandhi                                                         10   
                               Un cuento bilingüe para cada día = A bilingual ...             10   
                               Tintin en América                                               9   
                    30-39      La magia de la plastilina                                       7   
                               Pensar sistémico : una introducción al pensamie...              5   
                               Mi PC : actualización, configuración, mantenimi...              4   
                               La gestión de la tecnología : elementos fundame...              3   
                               El inventor de mamás                                            3   
                               El río : exploraciones y descubrimientos en la ...              3   
                               The avengers                                                    3   
                               Los increibles                                                  3   
                               Las mil y una noches                                            3   
                               Actualización, mantenimiento y solución de prob...              3   
...                                                                                          ...   
Honda   sra.        50-64      American english : Curso de inglés.                             4   
                               El cuerpo humano                                                4   
                               Algebra : con gráficos y 6523 ejercicios y prob...              3   
                               Breve historia de la educación                                  3   
                               Los modelos pedagógicos                                         3   
                               Lo máximo del vallenato joven Jorge Celedón, Si...              3   
                               Cuerpo humano                                                   2   
                               Bolt un perro fuera de serie                        

## Resultados finales

In [15]:
# mezcla_df.to_csv('???.csv', index=False)